#Modelo de ML con GridSearch

In [1]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Crear un conjunto de datos sintético
X, y = make_classification(n_samples=1000, n_features=20, random_state=42)

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir el modelo
model = RandomForestClassifier(random_state=42)

# Definir el grid de parámetros
param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [1, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Definir GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# Entrenar el modelo con GridSearchCV
grid_search.fit(X_train, y_train)

# Mostrar los mejores parámetros
print("Mejores hiperparámetros encontrados:", grid_search.best_params_)


Fitting 5 folds for each of 36 candidates, totalling 180 fits
Mejores hiperparámetros encontrados: {'max_depth': 20, 'min_samples_split': 5, 'n_estimators': 50}


#Predicción

In [2]:
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pickle

iris = load_iris()
model = GaussianNB()

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1)
model_f = model.fit(X_train, y_train)

print("Model score: ", model.score(X_train, y_train))
print("Test Accuracy: ", model.score(X_test, y_test))

pickle.dump(model_f, open("model_gb.pkl", "wb"))

Model score:  0.9629629629629629
Test Accuracy:  0.9333333333333333


In [3]:
%%writefile main.html
<!DOCTYPE html>
<html>
<body>
    <h2>Introduce 4 valores flotantes</h2>
    <form method="POST" action="result">
        <label for="float1">Float 1:</label><br>
        <input type="text" id="float1" name="float1"><br><br>

        <label for="float2">Float 2:</label><br>
        <input type="text" id="float2" name="float2"><br><br>

        <label for="float3">Float 3:</label><br>
        <input type="text" id="float3" name="float3"><br><br>

        <label for="float4">Float 4:</label><br>
        <input type="text" id="float4" name="float4"><br><br>

        <input type="submit" value="Enviar">
    </form>
</body>
</html>

Writing main.html


#Flask

Usaremos flask-ngrok para crear un servidor y proponer una URL publica

In [4]:
!pip install flask-ngrok
!pip install flask
!pip install pyngrok

In [5]:
import getpass
import os
import threading
from flask import Flask
from pyngrok import ngrok, conf

print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken")
conf.get_default().auth_token = getpass.getpass()

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken
··········


In [7]:
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
app = Flask(__name__, template_folder='/content')


print(ngrok.connect(5000).public_url)
run_with_ngrok(app)


@app.route("/")
def home():
    return render_template('main.html')

@app.route('/test')
def test():
    return "Flask is working!"

@app.route('/result', methods=['POST'])
def pred():
     # Obtener los valores del formulario
    float1 =  float(request.form.get('float1'))
    float2 =  float(request.form.get('float2'))
    float3 =  float(request.form.get('float3'))
    float4 =  float(request.form.get('float4'))
    p=np.array([float1,float2,float3,float4])

    loaded_model = pickle.load(open("model_gb.pkl", "rb"))
    # Definir los tipos de iris
    iris_types = {
        0: 'setosa',
        1: 'versicolor',
        2: 'virginica'
    }
    # Hacer predicciones
    prediction = model.predict([p])
    if prediction[0] == 0:
      label='setosa'
    elif prediction[0] == 1:
      label='versicolor'
    elif prediction[0] == 2:
      label='virginica'
    print(label)
    return label


app.run()

https://94cd-34-150-210-185.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://94cd-34-150-210-185.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [13/Sep/2024 05:16:53] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Sep/2024 05:16:53] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [13/Sep/2024 05:16:58] "POST /result HTTP/1.1" 200 -


virginica


INFO:werkzeug:127.0.0.1 - - [13/Sep/2024 05:17:08] "POST /result HTTP/1.1" 200 -


virginica
